Take your XGBoost skills to the next level by incorporating your models into two end-to-end machine learning pipelines. You'll learn how to tune the most important XGBoost hyperparameters efficiently within a pipeline, and get an introduction to some more advanced preprocessing techniques.

# 1- Review of pipelines using sklearn


video

# 2- Exploratory data analysis


<p>Before diving into the nitty gritty of pipelines and preprocessing, let&apos;s do some exploratory analysis of the original, unprocessed <a href="https://www.kaggle.com/c/house-prices-advanced-regression-techniques">Ames housing dataset</a>. When you worked with this data in previous chapters, we preprocessed it for you so you could focus on the core XGBoost concepts. In this chapter, you&apos;ll do the preprocessing yourself!</p>
<p>A smaller version of this original, unprocessed dataset has been pre-loaded into a <code>pandas</code> DataFrame called <code>df</code>. Your task is to explore <code>df</code> in the Shell and pick the option that is <strong>incorrect</strong>. The larger purpose of this exercise is to understand the kinds of transformations you will need to perform in order to be able to use XGBoost.</p>

In [1]:
import pandas as pd
# load data
df = pd.read_csv('datasets/ames_unprocessed_data.csv')


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 21 columns):
MSSubClass      1460 non-null int64
MSZoning        1460 non-null object
LotFrontage     1201 non-null float64
LotArea         1460 non-null int64
Neighborhood    1460 non-null object
BldgType        1460 non-null object
HouseStyle      1460 non-null object
OverallQual     1460 non-null int64
OverallCond     1460 non-null int64
YearBuilt       1460 non-null int64
Remodeled       1460 non-null int64
GrLivArea       1460 non-null int64
BsmtFullBath    1460 non-null int64
BsmtHalfBath    1460 non-null int64
FullBath        1460 non-null int64
HalfBath        1460 non-null int64
BedroomAbvGr    1460 non-null int64
Fireplaces      1460 non-null int64
GarageArea      1460 non-null int64
PavedDrive      1460 non-null object
SalePrice       1460 non-null int64
dtypes: float64(1), int64(15), object(5)
memory usage: 239.7+ KB


In [2]:
df.describe()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,Remodeled,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageArea,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,0.476712,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,0.613014,472.980137,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,0.499629,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.644666,213.804841,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,334.500000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,480.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,1.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,576.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,1.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,1418.000000,755000.000000


# 3- Encoding categorical columns I: LabelEncoder

<p>Now that you&apos;ve seen what will need to be done to get the housing data ready for XGBoost, let&apos;s go through the process step-by-step. </p>
<p>First, you will need to fill in missing values - as you saw previously, the column <code>LotFrontage</code> has many missing values. Then, you will need to encode any categorical columns in the dataset using one-hot encoding so that they are encoded numerically. You can watch <a href="https://campus.datacamp.com/courses/supervised-learning-with-scikit-learn/preprocessing-and-pipelines?ex=1">this video</a> from <a href="https://www.datacamp.com/courses/supervised-learning-with-scikit-learn">Supervised Learning with scikit-learn</a> for a refresher on the idea. </p>
<p>The data has five categorical columns: <code>MSZoning</code>, <code>PavedDrive</code>, <code>Neighborhood</code>, <code>BldgType</code>, and <code>HouseStyle</code>. Scikit-learn has a <a href="http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html">LabelEncoder</a> function that converts the values in each categorical column into integers. You&apos;ll practice using this here.</p>

<ul>
<li>Import <code>LabelEncoder</code> from <code>sklearn.preprocessing</code>.</li>
<li>Fill in missing values in the <code>LotFrontage</code> column with <code>0</code> using <code>.fillna()</code>.</li>
<li>Create a boolean mask for categorical columns. You can do this by checking for whether <code>df.dtypes</code> equals <code>object</code>.</li>
<li>Create a <code>LabelEncoder</code> object. You can do this in the same way you instantiate any scikit-learn estimator. </li>
<li>Encode all of the categorical columns into integers using <code>LabelEncoder()</code>. To do this, use the <code>.fit_transform()</code> method of <code>le</code> in the provided lambda function.</li>
</ul>

In [3]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Fill missing values with 0
df.LotFrontage = df['LotFrontage'].fillna(0)

# Create a boolean mask for categorical columns
categorical_mask = (df.dtypes == object)

# Get list of categorical column names
categorical_columns = df.columns[categorical_mask].tolist()

# Print the head of the categorical columns
print(df[categorical_columns].head())

# Create LabelEncoder object: le
le = LabelEncoder()

# Apply LabelEncoder to categorical columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))

# Print the head of the LabelEncoded categorical columns
print(df[categorical_columns].head())

  MSZoning Neighborhood BldgType HouseStyle PavedDrive
0       RL      CollgCr     1Fam     2Story          Y
1       RL      Veenker     1Fam     1Story          Y
2       RL      CollgCr     1Fam     2Story          Y
3       RL      Crawfor     1Fam     2Story          Y
4       RL      NoRidge     1Fam     2Story          Y
   MSZoning  Neighborhood  BldgType  HouseStyle  PavedDrive
0         3             5         0           5           2
1         3            24         0           2           2
2         3             5         0           5           2
3         3             6         0           5           2
4         3            15         0           5           2


# 4- Encoding categorical columns II: OneHotEncoder

<p>Okay - so you have your categorical columns encoded numerically. Can you now move onto using pipelines and XGBoost? Not yet! In the categorical columns of this dataset, there is no natural ordering between the entries. As an example: Using <code>LabelEncoder</code>, the <code>CollgCr</code> <code>Neighborhood</code> was encoded as <code>5</code>, while the <code>Veenker</code> <code>Neighborhood</code> was encoded as <code>24</code>, and <code>Crawfor</code> as <code>6</code>. Is <code>Veenker</code> &quot;greater&quot; than <code>Crawfor</code> and <code>CollgCr</code>? No - and allowing the model to assume this natural ordering may result in poor performance.</p>
<p>As a result, there is another step needed: You have to apply a one-hot encoding to create binary, or &quot;dummy&quot; variables. You can do this using scikit-learn&apos;s <a href="http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html">OneHotEncoder</a>.</p>

<ul>
<li>Import <code>OneHotEncoder</code> from <code>sklearn.preprocessing</code>.</li>
<li>Instantiate a <code>OneHotEncoder</code> object called <code>ohe</code>. Specify the keyword arguments <code>categorical_features=categorical_mask</code> and <code>sparse=False</code>.</li>
<li>Using its <code>.fit_transform()</code> method, apply the <code>OneHotEncoder</code> to <code>df</code> and save the result as <code>df_encoded</code>. The output will be a NumPy array.</li>
<li>Print the first 5 rows of <code>df_encoded</code>, and then the shape of <code>df</code> as well as <code>df_encoded</code> to compare the difference.</li>
</ul>

In [4]:
# Import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Create OneHotEncoder: ohe
ohe = OneHotEncoder(categorical_features=categorical_mask, sparse=False)

# Apply OneHotEncoder to categorical columns - output is no longer a dataframe: df_encoded
df_encoded = ohe.fit_transform(df)

# Print first 5 rows of the resulting dataset - again, this will no longer be a pandas dataframe
print(df_encoded[:5, :])

# Print the shape of the original DataFrame
print(df.shape)

# Print the shape of the transformed array
print(df_encoded.shape)

C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


[[0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 1.000e+00 6.000e+01 6.500e+01 8.450e+03
  7.000e+00 5.000e+00 2.003e+03 0.000e+00 1.710e+03 1.000e+00 0.000e+00
  2.000e+00 1.000e+00 3.000e+00 0.000e+00 5.480e+02 2.085e+05]
 [0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 1.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 

# 5- Encoding categorical columns III: DictVectorizer

<p>Alright, one final trick before you dive into pipelines. The two step process you just went through - <code>LabelEncoder</code> followed by <code>OneHotEncoder</code> - can be simplified by using a <a href="http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html">DictVectorizer</a>. </p>
<p>Using a <code>DictVectorizer</code> on a DataFrame that has been converted to a dictionary allows you to get label encoding as well as one-hot encoding in one go. </p>
<p>Your task is to work through this strategy in this exercise!</p>

<ul>
<li>Import <code>DictVectorizer</code> from <code>sklearn.feature_extraction</code>.</li>
<li>Convert <code>df</code> into a dictionary called <code>df_dict</code> using its <code>.to_dict()</code> method with <code>&quot;records&quot;</code> as the argument.</li>
<li>Instantiate a <code>DictVectorizer</code> object called <code>dv</code> with the keyword argument <code>sparse=False</code>.</li>
<li>Apply the <code>DictVectorizer</code> on <code>df_dict</code> by using its <code>.fit_transform()</code> method.</li>
<li>Hit &apos;Submit Answer&apos; to print the resulting first five rows and the vocabulary.</li>
</ul>

In [5]:
# Import DictVectorizer
from sklearn.feature_extraction import DictVectorizer

# Convert df into a dictionary: df_dict
df_dict = df.to_dict('records')

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False)

# Apply dv on df: df_encoded
df_encoded = dv.fit_transform(df_dict)

# Print the resulting first five rows
print(df_encoded[:5,:])

# Print the vocabulary
print(dv.vocabulary_)

[[3.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 2.000e+00 5.480e+02
  1.710e+03 1.000e+00 5.000e+00 8.450e+03 6.500e+01 6.000e+01 3.000e+00
  5.000e+00 5.000e+00 7.000e+00 2.000e+00 0.000e+00 2.085e+05 2.003e+03]
 [3.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 2.000e+00 4.600e+02
  1.262e+03 0.000e+00 2.000e+00 9.600e+03 8.000e+01 2.000e+01 3.000e+00
  2.400e+01 8.000e+00 6.000e+00 2.000e+00 0.000e+00 1.815e+05 1.976e+03]
 [3.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 2.000e+00 6.080e+02
  1.786e+03 1.000e+00 5.000e+00 1.125e+04 6.800e+01 6.000e+01 3.000e+00
  5.000e+00 5.000e+00 7.000e+00 2.000e+00 1.000e+00 2.235e+05 2.001e+03]
 [3.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 1.000e+00 6.420e+02
  1.717e+03 0.000e+00 5.000e+00 9.550e+03 6.000e+01 7.000e+01 3.000e+00
  6.000e+00 5.000e+00 7.000e+00 2.000e+00 1.000e+00 1.400e+05 1.915e+03]
 [4.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 2.000e+00 8.360e+02
  2.198e+03 1.000e+00 5.000e+00 1.426e+04 8.400e+01 6.000e+0

# 6- Preprocessing within a pipeline


<p>Now that you&apos;ve seen what steps need to be taken individually to properly process the Ames housing data, let&apos;s use the much cleaner and more succinct <code>DictVectorizer</code> approach and put it alongside an <code>XGBoostRegressor</code> inside of a scikit-learn pipeline.</p>

<ul>
<li>Import <code>DictVectorizer</code> from <code>sklearn.feature_extraction</code> and <code>Pipeline</code> from <code>sklearn.pipeline</code>.</li>
<li>Fill in any missing values in the <code>LotFrontage</code> column of <code>X</code> with <code>0</code>.</li>
<li>Complete the steps of the pipeline with <code>DictVectorizer(sparse=False)</code> for <code>&quot;ohe_onestep&quot;</code> and <code>xgb.XGBRegressor()</code> for <code>&quot;xgb_model&quot;</code>.</li>
<li>Create the pipeline using <code>Pipeline()</code> and <code>steps</code>.</li>
<li>Fit the <code>Pipeline</code>. Don&apos;t forget to convert <code>X</code> into a format that <code>DictVectorizer</code> understands by calling the <code>to_dict(&quot;records&quot;)</code> method on <code>X</code>.</li>
</ul>

In [9]:
# Import necessary modules
import xgboost as xgb
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline


X=df.drop('SalePrice', axis=1)
y=df['SalePrice']

# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

# Setup the pipeline steps: steps
steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor())]

# Create the pipeline: xgb_pipeline
xgb_pipeline = Pipeline(steps)

# Fit the pipeline
xgb_pipeline.fit(X.to_dict("records"), y)

[20:57:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Pipeline(memory=None,
         steps=[('ohe_onestep',
                 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=',
                                sort=True, sparse=False)),
                ('xgb_model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0,
                              importance_type='gain', learning_rate=0.1,
                              max_delta_step=0, max_depth=3, min_child_weight=1,
                              missing=None, n_estimators=100, n_jobs=1,
                              nthread=None, objective='reg:linear',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, seed=None, silent=None,
                              subsample=1, verbosity=1))],
         verbose=False)

# 7- Incorporating XGBoost into pipelines


video

# 8- Cross-validating your XGBoost model

In this exercise, you'll go one step further by using the pipeline you've created to preprocess and cross-validate your model.



<ul>
<li>Create a pipeline called <code>xgb_pipeline</code> using <code>steps</code>.</li>
<li>Perform 10-fold cross-validation using <a href="http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html"><code>cross_val_score()</code></a>. You&apos;ll have to pass in the pipeline, <code>X</code> (as a dictionary, using <code>.to_dict(&quot;records&quot;)</code>), <code>y</code>, the number of folds you want to use, and <code>scoring</code> (<code>&quot;neg_mean_squared_error&quot;</code>). </li>
<li>Print the 10-fold RMSE.</li>
</ul>

In [11]:
# Import necessary modules
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

# Setup the pipeline steps: steps
steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor(max_depth=2, objective="reg:linear"))]

# Create the pipeline: xgb_pipeline
xgb_pipeline = Pipeline(steps)

# Cross-validate the model
cross_val_scores = cross_val_score(xgb_pipeline, X.to_dict("records"), y, cv=10, scoring="neg_mean_squared_error")

# Print the 10-fold RMSE
print("10-fold RMSE: ", np.mean(np.sqrt(np.abs(cross_val_scores))))

C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:08:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:08:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:08:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:08:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:08:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:08:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:08:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:08:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\MohammedTaysser\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:08:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:08:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
10-fold RMSE:  30343.486551766466


# 9- Kidney disease case study I: Categorical Imputer

<p>You&apos;ll now continue your exploration of using pipelines with a dataset that requires significantly more wrangling. The <a href="https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease">chronic kidney disease dataset</a> contains both categorical and numeric features, but contains lots of missing values. The goal here is to predict who has chronic kidney disease given various blood indicators as features.</p>
<p>As Sergey mentioned in the video, you&apos;ll be introduced to a new library, <a href="https://github.com/pandas-dev/sklearn-pandas"><code>sklearn_pandas</code></a>, that allows you to chain many more processing steps inside of a pipeline than are currently supported in scikit-learn. Specifically, you&apos;ll be able to impute missing categorical values directly using the <code>Categorical_Imputer()</code> class in <code>sklearn_pandas</code>, and the <code>DataFrameMapper()</code> class to apply any arbitrary sklearn-compatible transformer on DataFrame columns, where the resulting output can be either a NumPy array or DataFrame.</p>
<p>We&apos;ve also created a transformer called a <code>Dictifier</code> that encapsulates converting a DataFrame using <code>.to_dict(&quot;records&quot;)</code> without you having to do it explicitly (and so that it works in a pipeline). Finally, we&apos;ve also provided the list of feature names in <code>kidney_feature_names</code>, the target name in <code>kidney_target_name</code>, the features in <code>X</code>, and the target in <code>y</code>.</p>
<p>In this exercise, your task is to apply the <code>CategoricalImputer</code> to impute all of the categorical columns in the dataset. You can refer to how the numeric imputation mapper was created as a template. Notice the keyword arguments <code>input_df=True</code> and <code>df_out=True</code>? This is so that you can work with DataFrames instead of arrays. By default, the transformers are passed a <code>numpy</code> array of the selected columns as input, and as a result, the output of the DataFrame mapper is also an array. Scikit-learn transformers have historically been designed to work with <code>numpy</code> arrays, not <code>pandas</code> DataFrames, even though their basic indexing interfaces are similar.</p>

In [8]:
#importing and processing 
import pandas as pd
names=['age',  'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo',
 'pcv', 'wc', 'rc', 'rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe',
 'ane', 'class']

kidney_data = pd.read_csv('datasets/kidney_disease_unprocessed_kaggle.csv', names=names)
X, y = kidney_data.iloc[:,:-1], kidney_data.iloc[:,-1]

kidney_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401 entries, id to 399
Data columns (total 25 columns):
age      392 non-null object
bp       389 non-null object
sg       354 non-null object
al       355 non-null object
su       352 non-null object
bgr      249 non-null object
bu       336 non-null object
sc       397 non-null object
sod      397 non-null object
pot      357 non-null object
hemo     382 non-null object
pcv      384 non-null object
wc       314 non-null object
rc       313 non-null object
rbc      349 non-null object
pc       331 non-null object
pcc      296 non-null object
ba       271 non-null object
htn      399 non-null object
dm       399 non-null object
cad      399 non-null object
appet    400 non-null object
pe       400 non-null object
ane      400 non-null object
class    401 non-null object
dtypes: object(25)
memory usage: 81.5+ KB


In [9]:
# Import necessary modules
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer

# Check number of nulls in each feature column
nulls_per_column = X.isnull().sum()
print(nulls_per_column)

# Create a boolean mask for categorical columns
categorical_feature_mask = X.dtypes == object

# Get list of categorical column names
categorical_columns = X.columns[categorical_feature_mask].tolist()

# Get list of non-categorical column names
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()

# Apply numeric imputer
numeric_imputation_mapper = DataFrameMapper(
                                            [([numeric_feature],Imputer(strategy="median")) for numeric_feature in non_categorical_columns],
                                            input_df=True,
                                            df_out=True
                                           )

# Apply categorical imputer
categorical_imputation_mapper = DataFrameMapper(
                                                [(category_feature, CategoricalImputer()) for category_feature in categorical_columns],
                                                input_df=True,
                                                df_out=True
                                               )

age        9
bp        12
sg        47
al        46
su        49
bgr      152
bu        65
sc         4
sod        4
pot       44
hemo      19
pcv       17
wc        87
rc        88
rbc       52
pc        70
pcc      105
ba       130
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
dtype: int64


In [10]:
kidney_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
age      400 non-null object
bp       400 non-null object
sg       400 non-null object
al       400 non-null object
su       400 non-null object
bgr      400 non-null object
bu       400 non-null object
sc       400 non-null object
sod      400 non-null object
pot      400 non-null object
hemo     400 non-null object
pcv      400 non-null object
wc       400 non-null object
rc       400 non-null object
rbc      400 non-null object
pc       400 non-null object
pcc      400 non-null object
ba       400 non-null object
htn      400 non-null object
dm       400 non-null object
cad      400 non-null object
appet    400 non-null object
pe       400 non-null object
ane      400 non-null object
class    400 non-null object
dtypes: object(25)
memory usage: 78.2+ KB


# 10- Kidney disease case study II: Feature Union

<p>Having separately imputed numeric as well as categorical columns, your task is now to use scikit-learn&apos;s <a href="http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html">FeatureUnion</a> to concatenate their results, which are contained in two separate transformer objects - <code>numeric_imputation_mapper</code>, and <code>categorical_imputation_mapper</code>, respectively.</p>
<p>You may have already encountered <code>FeatureUnion</code> in <a href="https://campus.datacamp.com/courses/machine-learning-with-the-experts-school-budgets/improving-your-model?ex=7">Machine Learning with the Experts: School Budgets</a>. Just like with pipelines, you have to pass it a list of <code>(string, transformer)</code> tuples, where the first half of each tuple is the name of the transformer.</p>

<ul>
<li>Import <code>FeatureUnion</code> from <code>sklearn.pipeline</code>.</li>
<li>Combine the results of <code>numeric_imputation_mapper</code> and <code>categorical_imputation_mapper</code> using <code>FeatureUnion()</code>, with the names <code>&quot;num_mapper&quot;</code> and <code>&quot;cat_mapper&quot;</code> respectively.</li>
</ul>

In [10]:
# Import FeatureUnion
from sklearn.pipeline import FeatureUnion

# Combine the numeric and categorical transformations
numeric_categorical_union = FeatureUnion([
                                          ("num_mapper", numeric_imputation_mapper),
                                          ("cat_mapper", categorical_imputation_mapper)
                                         ])

# 11- Kidney disease case study III: Full pipeline"

<p>It&apos;s time to piece together all of the transforms along with an <code>XGBClassifier</code> to build the full pipeline!</p>
<p>Besides the <code>numeric_categorical_union</code> that you created in the previous exercise, there are two other transforms needed: the <code>Dictifier()</code> transform which we created for you, and the <code>DictVectorizer()</code>. </p>
<p>After creating the pipeline, your task is to cross-validate it to see how well it performs.</p>

<ul>
<li>Create the pipeline using the <code>numeric_categorical_union</code>, <code>Dictifier()</code>, and <code>DictVectorizer(sort=False)</code> transforms, and <code>xgb.XGBClassifier()</code> estimator with <code>max_depth=3</code>. Name the transforms <code>&quot;featureunion&quot;</code>, <code>&quot;dictifier&quot;</code> <code>&quot;vectorizer&quot;</code>, and the estimator <code>&quot;clf&quot;</code>.</li>
<li>Perform 3-fold cross-validation on the <code>pipeline</code> using <code>cross_val_score()</code>. Pass it the pipeline, <code>pipeline</code>, the features, <code>kidney_data</code>, the outcomes, <code>y</code>. Also set <code>scoring</code> to <code>&quot;roc_auc&quot;</code> and <code>cv</code> to <code>3</code>.</li>
</ul>

In [ ]:

# Create full pipeline
pipeline = Pipeline([
                     ("featureunion", numeric_categorical_union),
                     ("dictifier", Dictifier()),
                     ("vectorizer", DictVectorizer(sort=False)),
                     ("clf", xgb.XGBClassifier(max_depth=3))
                    ])

# Perform cross-validation
cross_val_scores = cross_val_score(pipeline, kidney_data, y, scoring="roc_auc", cv=3)

# Print avg. AUC
print("3-fold AUC: ", np.mean(cross_val_scores))

# 12- Bringing it all together


<p>Alright, it&apos;s time to bring together everything you&apos;ve learned so far! In this final exercise of the course, you will combine your work from the previous exercises into one end-to-end XGBoost pipeline to really cement your understanding of preprocessing and pipelines in XGBoost.</p>
<p>Your work from the previous 3 exercises, where you preprocessed the data and set up your pipeline, has been pre-loaded. Your job is to perform a randomized search and identify the best hyperparameters.</p>

<ul>
<li>Set up the parameter grid to tune <code>&apos;clf__learning_rate&apos;</code> (from <code>0.05</code> to <code>1</code> in increments of <code>0.05</code>), <code>&apos;clf__max_depth&apos;</code> (from <code>3</code> to <code>10</code> in increments of <code>1</code>), and <code>&apos;clf__n_estimators&apos;</code> (from <code>50</code> to <code>200</code> in increments of <code>50</code>).</li>
<li>Using your <code>pipeline</code> as the estimator, perform 2-fold <code>RandomizedSearchCV</code> with an <code>n_iter</code> of <code>2</code>. Use <code>&quot;roc_auc&quot;</code> as the metric, and set <code>verbose</code> to <code>1</code> so the output is more detailed. Store the result in <code>randomized_roc_auc</code>.</li>
<li>Fit <code>randomized_roc_auc</code> to <code>X</code> and <code>y</code>.</li>
<li>Compute the best score and best estimator of <code>randomized_roc_auc</code>.</li>
</ul>

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

# Create the parameter grid
gbm_param_grid = {
    'clf__learning_rate': np.arange(.05, 1, .05),
    'clf__max_depth': np.arange(3,10, 1),
    'clf__n_estimators': np.arange(50, 200, 50)
}

# Perform RandomizedSearchCV
randomized_roc_auc = RandomizedSearchCV(estimator=pipeline,
                                        param_distributions=gbm_param_grid,
                                        n_iter=2, scoring='roc_auc', cv=2, verbose=1)

# Fit the estimator
randomized_roc_auc.fit(X, y)

# Compute metrics
print(randomized_roc_auc.best_score_)
print(randomized_roc_auc.best_estimator_)